In [1]:
import numpy as np

from pyboolnet.external.bnet2primes import bnet_text2primes
from pyboolnet.state_transition_graphs import primes2stg

from transition_matrix import get_transition_matrix
from scc_dags import get_scc_dag, get_ordered_states, get_attractor_states
from matrix_operations import nsquare, compress_matrix, expand_matrix, get_rms_diff, get_dkl
from grouping import sd_grouping
from reachability import get_convergence_matrix

np.set_printoptions(linewidth=1000, precision=3, suppress=True)

In [2]:
bnet = """
A, A | B & C
B, B & !C
C, B & !C | !C & !D | !B & C & D
D, !A & !B & !C & !D | !A & C & D
"""

update = "asynchronous"

DEBUG = True

In [3]:
primes = bnet_text2primes(bnet)
primes = {key: primes[key] for key in sorted(primes)}
stg = primes2stg(primes, update)

scc_dag = get_scc_dag(stg)

scc_indices = get_ordered_states(scc_dag, as_indices=True, DEBUG=DEBUG)
attractor_indices = get_attractor_states(scc_dag, as_indices=True, DEBUG=DEBUG)

print(scc_indices)
print(attractor_indices)

[[9], [5, 7], [13, 15], [4, 6], [11], [12, 14], [0, 1, 2], [3], [8, 10]]
[[0, 1, 2], [3], [8, 10]]


In [4]:
T = get_transition_matrix(stg, update=update, DEBUG=DEBUG)

print("T")
print(T)

T_inf = nsquare(T, 20, DEBUG=DEBUG)

print("T_inf")
print(T_inf)

T
[[0.5  0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.75 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.   0.75 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.75 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.5  0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.25 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25 0.   0.25 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.25]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.75 0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.75 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.75 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.75 0.   0.   0.   0.  ]
 [

In [5]:
sd_indices = sd_grouping(bnet, DEBUG=DEBUG)

print(sd_indices)

Tsd = compress_matrix(T, sd_indices, DEBUG=DEBUG)

print("Tsd")
print(Tsd)

Tsd_inf = nsquare(Tsd, 20, DEBUG=DEBUG)

print("Tsd_inf")
print(Tsd_inf)

Tsd_inf_expanded = expand_matrix(Tsd_inf, sd_indices, DEBUG=DEBUG)

print("Tsd_inf_expanded")
print(Tsd_inf_expanded)

[[4, 5, 6, 7], [], [0, 1, 2], [3], [12, 14], [8, 10], [13, 15], [9, 11]]
Tsd
[[0.75  0.062 0.062 0.062 0.    0.062 0.   ]
 [0.    1.    0.    0.    0.    0.    0.   ]
 [0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.875 0.125 0.    0.   ]
 [0.    0.    0.    0.    1.    0.    0.   ]
 [0.    0.    0.    0.25  0.    0.625 0.125]
 [0.    0.    0.    0.    0.25  0.    0.75 ]]
Tsd_inf
[[0.   0.25 0.25 0.   0.5  0.   0.  ]
 [0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.  ]]
Tsd_inf_expanded
[[0.333 0.333 0.333 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.333 0.333 0.333 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.333 0.333 0.333 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.  

In [6]:
T_convergence, _, _ = get_convergence_matrix(T_inf, scc_indices, attractor_indices, DEBUG=DEBUG)
Tsd_convergence, _, _ = get_convergence_matrix(Tsd_inf_expanded, scc_indices, attractor_indices, DEBUG=DEBUG)

print("T_convergence")
print(T_convergence)

print("Tsd_convergence")
print(Tsd_convergence)

T_convergence
[[0.  0.  1. ]
 [0.3 0.2 0.5]
 [0.1 0.4 0.5]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.5 0.  0.5]
 [0.5 0.  0.5]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]]
Tsd_convergence
[[0.   0.   1.  ]
 [0.25 0.25 0.5 ]
 [0.25 0.25 0.5 ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.25 0.25 0.5 ]
 [0.25 0.25 0.5 ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]]


In [7]:
rmsd = get_rms_diff(T_convergence, Tsd_convergence, partial=True, DEBUG=DEBUG)
dkl = get_dkl(T_convergence, Tsd_convergence, partial=True, DEBUG=DEBUG)

print("rmsd")
print(rmsd)

print("dkl")
print(dkl)

rmsd
0.10000000000000003
dkl
0.7995873158461685
